In [4]:
import os
%pwd
os.chdir("../")
%pwd

'e:\\reviewprediction'

In [5]:
import pandas as pd
final_dataframe=pd.read_csv("artifacts/data_ingestion/final_dataframe.csv")

In [6]:
labels = final_dataframe['review_score']
final_dataframe.drop('review_score', axis=1, inplace=True)

In [7]:
''' creating a new variable called 'labels' which contain the edited class polarity. This column should be removed from the original table.'''

" creating a new variable called 'labels' which contain the edited class polarity. This column should be removed from the original table."

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(final_dataframe, labels, stratify=labels, test_size=0.2, random_state=0)

In [9]:
print('Train data:', X_train.shape, y_train.shape)
print('Test data:', X_test.shape, y_test.shape)

Train data: (79787, 32) (79787,)
Test data: (19947, 32) (19947,)


In [10]:
final_dataframe.shape

(99734, 32)

In [11]:
'''Creating a function which will encode the categorical features using response coding.'''

'Creating a function which will encode the categorical features using response coding.'

In [12]:
def train_response(frame):
  f1 = frame[frame.iloc[:,1] == 0]
  f2 = frame[frame.iloc[:,1] == 1]
  global dict_frame, dict_f1, dict_f2
  dict_frame = dict(frame.iloc[:,0].value_counts())
  dict_f1 = dict(f1.iloc[:,0].value_counts())
  dict_f2 = dict(f2.iloc[:,0].value_counts())
  state_0, state_1 = [],[],
  for i in range(len(frame)):
    if frame.iloc[:,1][i] == 0:
      state_0.append(dict_f1.get(frame.iloc[:,0][i],0)/dict_frame[frame.iloc[:,0][i]])
      state_1.append(float(1-state_0[-1]))
    else:
      state_1.append(dict_f2.get(frame.iloc[:,0][i],0)/dict_frame[frame.iloc[:,0][i]])
      state_0.append(float(1-state_1[-1])) 
  df3 = pd.DataFrame({'State_0':state_0, 'State_1':state_1})
  return df3.to_numpy()

def test_response(test):
  t_state_0, t_state_1 = [],[]
  for i in range(len(test)):
    if dict_frame.get(test[i]):
      t_state_0.append(dict_f1.get(test[i],0)/dict_frame.get(test[i]))
      t_state_1.append(dict_f2.get(test[i],0)/dict_frame.get(test[i]))
    else:
      t_state_0.append(0.5)
      t_state_1.append(0.5)
  df4 = pd.DataFrame({'State_0':t_state_0, 'State_1':t_state_1})
  return df4.to_numpy()  

In [13]:
'''We shall encode the categorical features which contain fewer categories using one hot encoding and encode them using response coding if there are too many categories
We won't be using one hot encoding when there are too many categories because it would add too many extra dimnesions and in turn lead to curse of dimensionality.'''

"We shall encode the categorical features which contain fewer categories using one hot encoding and encode them using response coding if there are too many categories\nWe won't be using one hot encoding when there are too many categories because it would add too many extra dimnesions and in turn lead to curse of dimensionality."

In [14]:
X_train_resp_prod_cat = train_response(pd.concat([X_train['product_category_name'], y_train], axis=1).reset_index(drop=True))
X_test_resp_prod_cat = test_response(X_test['product_category_name'].values)

In [15]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, OrdinalEncoder
ohe_order_item = OneHotEncoder()
ohe_order_item.fit(X_train['order_item_id'].values.reshape(-1,1))
X_train_order_item = ohe_order_item.transform(X_train['order_item_id'].values.reshape(-1,1)).toarray()
X_test_order_item = ohe_order_item.transform(X_test['order_item_id'].values.reshape(-1,1)).toarray()

In [16]:
X_train_resp_payment_seq = train_response(pd.concat([X_train['payment_sequential'], y_train], axis=1).reset_index(drop=True))
X_test_resp_payment_seq = test_response(X_test['payment_sequential'].values)


In [17]:
ohe_payment_type = OneHotEncoder()
ohe_payment_type.fit(X_train['payment_type'].values.reshape(-1,1))
X_train_payment_type = ohe_payment_type.transform(X_train['payment_type'].values.reshape(-1,1)).toarray()
X_test_payment_type = ohe_payment_type.transform(X_test['payment_type'].values.reshape(-1,1)).toarray()

In [18]:
import numpy as np
enc_price = OrdinalEncoder()
enc_price.fit(X_train['price_category'].values.reshape(-1,1))
enc_price.categories_ = [np.array([ 'cheap', 'affordable', 'expensive'], dtype=object)]
X_train_cat_price = enc_price.transform(X_train['price_category'].values.reshape(-1,1))
X_test_cat_price = enc_price.transform(X_test['price_category'].values.reshape(-1,1))

In [19]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79787 entries, 1457 to 96227
Data columns (total 32 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   order_item_id                         79787 non-null  int64  
 1   price                                 79787 non-null  float64
 2   freight_value                         79787 non-null  float64
 3   product_category_name                 79787 non-null  object 
 4   product_name_lenght                   79787 non-null  float64
 5   product_description_lenght            79787 non-null  float64
 6   product_photos_qty                    79787 non-null  float64
 7   product_weight_g                      79787 non-null  float64
 8   product_length_cm                     79787 non-null  float64
 9   product_height_cm                     79787 non-null  float64
 10  product_width_cm                      79787 non-null  float64
 11  seller_zip_code_p

In [20]:
X_train['review_comment_title']

1457     NaN
93284    NaN
67236    NaN
45770    NaN
37857    NaN
        ... 
14861    NaN
41063    NaN
56274    NaN
10446    NaN
96227    NaN
Name: review_comment_title, Length: 79787, dtype: object

In [21]:
X_test

,order_item_id,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,...,review_answer_timestamp,payment_sequential,payment_type,payment_installments,payment_value,purchase-delivery difference,estimated-actual delivery difference,price_category,purchase_delivery_diff_per_price,review_availability
96913,1,2990.00,194.55,agro_industry_and_commerce,42.0,505.0,1.0,30000.0,40.0,105.0,...,2017-12-20 18:26:41,1,credit_card,10,3184.55,52,-23,expensive,0.017391,1
86420,1,249.90,60.30,cool_stuff,29.0,266.0,3.0,5650.0,73.0,73.0,...,2017-10-25 18:38:13,1,credit_card,3,310.20,18,16,expensive,0.072029,1
81441,1,164.99,23.73,auto,53.0,578.0,3.0,1350.0,30.0,20.0,...,2018-04-24 23:39:27,1,credit_card,10,188.72,8,21,expensive,0.048488,1
80197,1,252.00,46.22,housewares,43.0,1002.0,3.0,10150.0,89.0,15.0,...,2018-02-02 16:44:17,1,credit_card,6,298.22,18,7,expensive,0.071429,1
69859,1,169.00,12.68,baby,41.0,565.0,1.0,450.0,16.0,12.0,...,2017-09-07 00:47:25,1,credit_card,3,181.68,6,13,expensive,0.035503,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34027,1,119.90,20.92,watches_gifts,54.0,424.0,2.0,1850.0,23.0,2.0,...,2017-04-23 06:38:08,1,credit_card,4,140.82,11,22,affordable,0.091743,0
25087,1,219.99,71.75,bed_bath_table,59.0,602.0,1.0,3100.0,28.0,20.0,...,2018-08-09 19:23:48,1,credit_card,1,291.74,8,14,expensive,0.036365,1
44565,1,26.90,10.96,fixed_telephony,52.0,401.0,11.0,100.0,19.0,3.0,...,2017-03-31 15:17:04,1,boleto,1,37.86,7,10,cheap,0.260223,0
72761,2,25.00,12.65,baby,60.0,865.0,1.0,500.0,23.0,23.0,...,2018-07-19 11:30:21,1,boleto,1,70.61,5,10,cheap,0.200000,0


In [22]:
#clean it using different preprocessing techniques.
#pip install spacy



In [23]:
import spacy
import regex as re
sp = spacy.load('pt_core_news_sm')
all_stopwords = sp.Defaults.stop_words

def process_texts(texts): 

    processed_text = []
    dates = '^([0]?[1-9]|[1|2][0-9]|[3][0|1])[./-]([0]?[1-9]|[1][0-2])[./-]([0-9]{4}|[0-9]{2})$'
    
    for text in texts:
        text = re.sub(r'\r\n|\r|\n', ' ', text) 
        text = re.sub(r'^https?:\/\/.*[\r\n]*', ' ', text) 
        text = re.sub(dates, ' ', text) 
        text = re.sub('[ \t]+$', '', text)
        text = re.sub('\W', ' ', text)
        text = re.sub('[0-9]+', ' ', text)
        text = re.sub('\s+', ' ', text)
        text = ' '.join(e for e in text.split() if e.lower() not in all_stopwords) 
        processed_text.append(text.lower().strip())
        
    return processed_text

In [24]:
X_train_comment_preprocess = process_texts(X_train['review_comment_message'])
X_test_comment_preprocess = process_texts(X_test['review_comment_message'])


In [25]:
#pip install sentence-transformers

from sentence_transformers import SentenceTransformer

model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model = SentenceTransformer(model_name)

e:\reviewprediction\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
len(X_train_comment_preprocess)

79787

In [27]:
len(X_train['review_comment_message'])

79787

In [28]:
encoded_train_reviews = model.encode(X_train_comment_preprocess)

# Verify encoded review length (should be equal to len(X_train))
print(len(encoded_train_reviews))

79787


In [29]:
# **Extract a single value for each review (assuming first element represents embedding):**
extracted_reviews = [review[0] for review in encoded_train_reviews]  # Assuming first element represents the review embedding

# Assign the extracted reviews to the DataFrame column
X_train = X_train.assign(embedded_review_comment_message=extracted_reviews)

In [30]:
encoded_test_reviews=model.encode(X_test_comment_preprocess)

In [31]:
# **Extract a single value for each review (assuming first element represents embedding):**
extracted_test_reviews = [review[0] for review in encoded_test_reviews]  # Assuming first element represents the review embedding

# Assign the extracted reviews to the DataFrame column
X_test = X_test.assign(embedded_review_comment_message=extracted_test_reviews)

In [32]:
X_test['embedded_review_comment_message']

96913    0.036108
86420   -0.053768
81441   -0.060682
80197    0.047778
69859   -0.032257
           ...   
34027    0.029497
25087   -0.040814
44565    0.029497
72761    0.029497
72297   -0.056857
Name: embedded_review_comment_message, Length: 19947, dtype: float32

In [33]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79787 entries, 1457 to 96227
Data columns (total 33 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   order_item_id                         79787 non-null  int64  
 1   price                                 79787 non-null  float64
 2   freight_value                         79787 non-null  float64
 3   product_category_name                 79787 non-null  object 
 4   product_name_lenght                   79787 non-null  float64
 5   product_description_lenght            79787 non-null  float64
 6   product_photos_qty                    79787 non-null  float64
 7   product_weight_g                      79787 non-null  float64
 8   product_length_cm                     79787 non-null  float64
 9   product_height_cm                     79787 non-null  float64
 10  product_width_cm                      79787 non-null  float64
 11  seller_zip_code_p

In [34]:
from sklearn.preprocessing import  StandardScaler

In [35]:
strn = StandardScaler()
strn.fit(X_train[['price','freight_value','product_photos_qty','product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm', 'payment_value','purchase-delivery difference','estimated-actual delivery difference','purchase_delivery_diff_per_price']])
X_train_strn = strn.transform(X_train[['price','freight_value','product_photos_qty','product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm', 'payment_value','purchase-delivery difference','estimated-actual delivery difference','purchase_delivery_diff_per_price']])
X_test_strn = strn.transform(X_test[['price','freight_value','product_photos_qty','product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm', 'payment_value','purchase-delivery difference','estimated-actual delivery difference','purchase_delivery_diff_per_price']])


In [36]:
#concatenate all the features which are necessary for the machine learning task.

In [37]:
X_train_final = np.concatenate((X_train_strn,X_train_resp_prod_cat,X_train_order_item,
       X_train_resp_payment_seq,X_train_payment_type,X_train_cat_price,X_train['review_availability'].values.reshape(-1,1),
       np.vstack(X_train['embedded_review_comment_message'].values)), axis=1)

X_test_final = np.concatenate((X_test_strn,X_test_resp_prod_cat, X_test_order_item,
       X_test_resp_payment_seq,X_test_payment_type,X_test_cat_price,X_test['review_availability'].values.reshape(-1,1),
       np.vstack(X_test['embedded_review_comment_message'].values)), axis=1)

In [38]:
print(X_train_final.shape)
print(X_test_final.shape)

(79787, 33)
(19947, 33)


In [39]:
y_test.shape

(19947,)

In [40]:
X_test_final.shape

(19947, 33)

In [41]:
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix


X_train = X_train_final
X_test = X_test_final
y_train = y_train
y_test = y_test

# Define the parameter grid 
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [5, 10, 15],
    "min_samples_split": range(2, 11),
    "min_samples_leaf": range(1, 5)
}

# Use RandomizedSearchCV for potentially faster exploration
random_search = RandomizedSearchCV(RandomForestClassifier(random_state=42),
                                     param_grid, scoring="f1_macro", cv=3, n_iter=50)
random_search.fit(X_train, y_train)

# Print the best parameters found by randomized search
print("\nBest parameters:", random_search.best_params_)

# Train the random forest classifier with the best parameters
best_rf_clf = RandomForestClassifier(**random_search.best_params_, random_state=42)
best_rf_clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_rf_clf.predict(X_test)

# Evaluate the model's performance
print("\nF1 score (macro):", f1_score(y_test, y_pred, average="macro"))
print("\nClassification report:\n", classification_report(y_test, y_pred))
print("\nConfusion matrix:\n", confusion_matrix(y_test, y_pred))

# Training and prediction time estimation
import time

start_time = time.time()
best_rf_clf.fit(X_train, y_train)
training_time = time.time() - start_time
print("\nTraining time:", training_time, "seconds")

start_time = time.time()
y_pred = best_rf_clf.predict(X_test)
prediction_time = time.time() - start_time
print("\nPrediction time:", prediction_time, "seconds")



Best parameters: {'n_estimators': 200, 'min_samples_split': 6, 'min_samples_leaf': 1, 'max_depth': 15}

F1 score (macro): 0.7366630937092551

Classification report:
               precision    recall  f1-score   support

           0       0.75      0.45      0.57      4387
           1       0.86      0.96      0.91     15560

    accuracy                           0.85     19947
   macro avg       0.81      0.71      0.74     19947
weighted avg       0.84      0.85      0.83     19947


Confusion matrix:
 [[ 1987  2400]
 [  647 14913]]

Training time: 48.897183656692505 seconds

Prediction time: 0.6900532245635986 seconds


In [42]:
try:
  from xgboost import XGBClassifier
  print("XGBoost module is available.")
except ImportError:
  print("XGBoost module is not available. Please install it using 'pip install xgboost'.")


XGBoost module is available.


In [43]:
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from lightgbm import LGBMClassifier

# Use your pre-processed data
X_train_final = X_train_final
X_test_final = X_test_final
y_train = y_train
y_test = y_test

# Define the parameter grid (adjust parameters and their ranges as needed)
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [5, 10, 15],
    "learning_rate": [0.1, 0.01],  # LightGBM-specific parameter
    "min_child_samples": range(1, 11)  # LightGBM-specific parameter
}

# Use RandomizedSearchCV for potentially faster exploration
random_search = RandomizedSearchCV(LGBMClassifier(random_state=42),
                                     param_grid, scoring="f1_macro", cv=3, n_iter=50)
random_search.fit(X_train_final, y_train)

# Print the best parameters found by randomized search
print("\nBest parameters:", random_search.best_params_)

# Train the LightGBM classifier with the best parameters
best_lgbm_clf = LGBMClassifier(**random_search.best_params_, random_state=42)
best_lgbm_clf.fit(X_train_final, y_train)

# Make predictions on the test set
y_pred = best_lgbm_clf.predict(X_test_final)

# Evaluate the model's performance
print("\nF1 score (macro):", f1_score(y_test, y_pred, average="macro"))
print("\nClassification report:\n", classification_report(y_test, y_pred))
print("\nConfusion matrix:\n", confusion_matrix(y_test, y_pred))

# Training and prediction time estimation
# (Note: Actual times may vary depending on hardware and data size)
import time

start_time = time.time()
best_lgbm_clf.fit(X_train_final, y_train)
training_time = time.time() - start_time
print("\nTraining time:", training_time, "seconds")

start_time = time.time()
y_pred = best_lgbm_clf.predict(X_test_final)
prediction_time = time.time() - start_time
print("\nPrediction time:", prediction_time, "seconds")


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 41491, number of negative: 11700
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004646 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2255
[LightGBM] [Info] Number of data points in the train set: 53191, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.780038 -> initscore=1.265888
[LightGBM] [Info] Start training from score 1.265888
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you did

In [44]:
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from xgboost import XGBClassifier

# Use your pre-processed data
X_train_final = X_train_final
X_test_final = X_test_final
y_train = y_train
y_test = y_test

# Define the parameter grid (adjust parameters and their ranges as needed)
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [5, 10, 15],
    "min_child_weight": range(1, 11),  # XGBoost-specific parameter
    "learning_rate": [0.1, 0.01],  # XGBoost-specific parameter
}

# Use RandomizedSearchCV for potentially faster exploration
random_search = RandomizedSearchCV(XGBClassifier(random_state=42),
                                     param_grid, scoring="f1_macro", cv=3, n_iter=50)
random_search.fit(X_train_final, y_train)

# Print the best parameters found by randomized search
print("\nBest parameters:", random_search.best_params_)

# Train the XGBoost classifier with the best parameters
best_xgb_clf = XGBClassifier(**random_search.best_params_, random_state=42)
best_xgb_clf.fit(X_train_final, y_train)

# Make predictions on the test set
y_pred = best_xgb_clf.predict(X_test_final)

# Evaluate the model's performance
print("\nF1 score (macro):", f1_score(y_test, y_pred, average="macro"))
print("\nClassification report:\n", classification_report(y_test, y_pred))
print("\nConfusion matrix:\n", confusion_matrix(y_test, y_pred))

# Training and prediction time estimation
# (Note: Actual times may vary depending on hardware and data size)
import time

start_time = time.time()
best_xgb_clf.fit(X_train_final, y_train)
training_time = time.time() - start_time
print("\nTraining time:", training_time, "seconds")

start_time = time.time()
y_pred = best_xgb_clf.predict(X_test_final)
prediction_time = time.time() - start_time
print("\nPrediction time:", prediction_time, "seconds")



Best parameters: {'n_estimators': 300, 'min_child_weight': 4, 'max_depth': 5, 'learning_rate': 0.1}

F1 score (macro): 0.7432196423471432

Classification report:
               precision    recall  f1-score   support

           0       0.76      0.47      0.58      4387
           1       0.86      0.96      0.91     15560

    accuracy                           0.85     19947
   macro avg       0.81      0.71      0.74     19947
weighted avg       0.84      0.85      0.84     19947


Confusion matrix:
 [[ 2053  2334]
 [  665 14895]]

Training time: 2.385098457336426 seconds

Prediction time: 0.06317281723022461 seconds


In [48]:
X_test_cat_price

array([[2.],
       [2.],
       [2.],
       ...,
       [0.],
       [0.],
       [1.]])

In [51]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

# Assuming you have your pre-processed data
#X_train_final1, X_test_final1, y_train1, y_test1 = train_test_split(X_train_final, y_train, test_size=0.2)

# Define and train the XGBClassifier model
model = XGBClassifier(n_estimators=300, min_child_weight=4, max_depth=5, learning_rate=0.1)
model.fit(X_train_final, y_train)

# Make predictions on the test set
predictions = model.predict(X_test_final)

# Evaluate model accuracy (using sklearn.metrics)
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)


Accuracy: 0.8496515766781972


In [52]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [54]:
from reviewprediction.constants import *
from reviewprediction.utils.common import read_yaml, create_directories

In [55]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [71]:
import os
from reviewprediction import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, OrdinalEncoder
import numpy as np
import spacy
import regex as re
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import  StandardScaler
import json

In [69]:
import pandas as pd
import json

# Assuming X_train_final and X_test_final are 2D NumPy arrays and y_train, y_test are 1D NumPy arrays

# Load the root directory from your config
root_dir = self.config.root_dir

# Construct file paths using the root directory
train_data_path = os.path.join(root_dir, "train_data.json")
test_data_path = os.path.join(root_dir, "test_data.json")

# Combine features and label
train_data = []
for features, label in zip(X_train_final, y_train):
    train_data.append({'features': features.tolist(), 'label': label})

test_data = []
for features, label in zip(X_test_final, y_test):
    test_data.append({'features': features.tolist(), 'label': label})

# Save to JSON files
with open(train_data_path, "w") as f:
    json.dump(train_data, f)

with open(test_data_path, "w") as f:
    json.dump(test_data, f)


AttributeError: 'list' object has no attribute 'tolist'

In [ ]:
import pandas as pd
import json

# Define the file paths (replace with your actual paths)
train_data_path = "arifacts/train_data.json"
test_data_path = "artifacts/test_data.json"

# Load the JSON data
try:
    with open(train_data_path, "r") as f:
        train_data = json.load(f)
    with open(test_data_path, "r") as f:
        test_data = json.load(f)
except FileNotFoundError:
    print("Error: JSON files not found at specified paths.")
else:
    print("Train and test data loaded from JSON files.")

# Extract features and labels
X_train_final = []
y_train1 = []
for item in train_data:
    X_train_final.append(item["features"])
    y_train1.append(item["label"])

X_test_final = []
y_test1 = []
for item in test_data:
    X_test_final.append(item["features"])
    y_test1 = [item["label"] for item in test_data]  # Shortened list comprehension

# Convert features back to NumPy arrays (if needed)
X_train_final_6969 = np.array(X_train_final)
X_test_final_6969 = np.array(X_test_final)

# Now you have X_train_final, X_test_final, y_train, and y_test in their original format
print("Data extracted from JSON files.")


In [78]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    
   
    def complete_transformation(self):
            final_dataframe = pd.read_csv(self.config.data_path)
            labels = final_dataframe['review_score']
            final_dataframe.drop('review_score', axis=1, inplace=True)
            X_train, X_test, y_train, y_test = train_test_split(final_dataframe, labels, stratify=labels, test_size=0.2, random_state=0)
            def train_response(frame):
                    f1 = frame[frame.iloc[:,1] == 0]
                    f2 = frame[frame.iloc[:,1] == 1]
                    global dict_frame, dict_f1, dict_f2
                    dict_frame = dict(frame.iloc[:,0].value_counts())
                    dict_f1 = dict(f1.iloc[:,0].value_counts())
                    dict_f2 = dict(f2.iloc[:,0].value_counts())
                    state_0, state_1 = [],[],
                    for i in range(len(frame)):
                        if frame.iloc[:,1][i] == 0:
                                state_0.append(dict_f1.get(frame.iloc[:,0][i],0)/dict_frame[frame.iloc[:,0][i]])
                                state_1.append(float(1-state_0[-1]))
                        else:
                                state_1.append(dict_f2.get(frame.iloc[:,0][i],0)/dict_frame[frame.iloc[:,0][i]])
                                state_0.append(float(1-state_1[-1])) 
                    df3 = pd.DataFrame({'State_0':state_0, 'State_1':state_1})
                    return df3.to_numpy()

            def test_response(test):
                    t_state_0, t_state_1 = [],[]
                    for i in range(len(test)):
                        if dict_frame.get(test[i]):
                            t_state_0.append(dict_f1.get(test[i],0)/dict_frame.get(test[i]))
                            t_state_1.append(dict_f2.get(test[i],0)/dict_frame.get(test[i]))
                        else:
                            t_state_0.append(0.5)
                            t_state_1.append(0.5)
                    df4 = pd.DataFrame({'State_0':t_state_0, 'State_1':t_state_1})
                    return df4.to_numpy() 
            
            X_train_resp_prod_cat = train_response(pd.concat([X_train['product_category_name'], y_train], axis=1).reset_index(drop=True))
            X_test_resp_prod_cat = test_response(X_test['product_category_name'].values)


            ohe_order_item = OneHotEncoder()
            ohe_order_item.fit(X_train['order_item_id'].values.reshape(-1,1))
            X_train_order_item = ohe_order_item.transform(X_train['order_item_id'].values.reshape(-1,1)).toarray()
            X_test_order_item = ohe_order_item.transform(X_test['order_item_id'].values.reshape(-1,1)).toarray()
            X_train_resp_payment_seq = train_response(pd.concat([X_train['payment_sequential'], y_train], axis=1).reset_index(drop=True))
            X_test_resp_payment_seq = test_response(X_test['payment_sequential'].values)

            ohe_payment_type = OneHotEncoder()
            ohe_payment_type.fit(X_train['payment_type'].values.reshape(-1,1))
            X_train_payment_type = ohe_payment_type.transform(X_train['payment_type'].values.reshape(-1,1)).toarray()
            X_test_payment_type = ohe_payment_type.transform(X_test['payment_type'].values.reshape(-1,1)).toarray()

            enc_price = OrdinalEncoder()
            enc_price.fit(X_train['price_category'].values.reshape(-1,1))
            enc_price.categories_ = [np.array([ 'cheap', 'affordable', 'expensive'], dtype=object)]
            X_train_cat_price = enc_price.transform(X_train['price_category'].values.reshape(-1,1))
            X_test_cat_price = enc_price.transform(X_test['price_category'].values.reshape(-1,1))


            sp = spacy.load('pt_core_news_sm')
            all_stopwords = sp.Defaults.stop_words

            def process_texts(texts): 

                processed_text = []
                dates = '^([0]?[1-9]|[1|2][0-9]|[3][0|1])[./-]([0]?[1-9]|[1][0-2])[./-]([0-9]{4}|[0-9]{2})$'
                
                for text in texts:
                    text = re.sub(r'\r\n|\r|\n', ' ', text) 
                    text = re.sub(r'^https?:\/\/.*[\r\n]*', ' ', text) 
                    text = re.sub(dates, ' ', text) 
                    text = re.sub('[ \t]+$', '', text)
                    text = re.sub('\W', ' ', text)
                    text = re.sub('[0-9]+', ' ', text)
                    text = re.sub('\s+', ' ', text)
                    text = ' '.join(e for e in text.split() if e.lower() not in all_stopwords) 
                    processed_text.append(text.lower().strip())
                    
                return processed_text


            X_train_comment_preprocess = process_texts(X_train['review_comment_message'])
            X_test_comment_preprocess = process_texts(X_test['review_comment_message'])


            model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
            model = SentenceTransformer(model_name)
            encoded_train_reviews = model.encode(X_train_comment_preprocess)
            extracted_reviews = [review[0] for review in encoded_train_reviews]
            X_train = X_train.assign(embedded_review_comment_message=extracted_reviews)

            encoded_test_reviews=model.encode(X_test_comment_preprocess)
            extracted_test_reviews = [review[0] for review in encoded_test_reviews]
            X_test = X_test.assign(embedded_review_comment_message=extracted_test_reviews)


            strn = StandardScaler()
            strn.fit(X_train[['price','freight_value','product_photos_qty','product_weight_g', 'product_length_cm',
                'product_height_cm', 'product_width_cm', 'payment_value','purchase-delivery difference','estimated-actual delivery difference','purchase_delivery_diff_per_price']])
            X_train_strn = strn.transform(X_train[['price','freight_value','product_photos_qty','product_weight_g', 'product_length_cm',
                'product_height_cm', 'product_width_cm', 'payment_value','purchase-delivery difference','estimated-actual delivery difference','purchase_delivery_diff_per_price']])
            X_test_strn = strn.transform(X_test[['price','freight_value','product_photos_qty','product_weight_g', 'product_length_cm',
                'product_height_cm', 'product_width_cm', 'payment_value','purchase-delivery difference','estimated-actual delivery difference','purchase_delivery_diff_per_price']])


            X_train_final = np.concatenate((X_train_strn,X_train_resp_prod_cat,X_train_order_item,
                X_train_resp_payment_seq,X_train_payment_type,X_train_cat_price,X_train['review_availability'].values.reshape(-1,1),
                np.vstack(X_train['embedded_review_comment_message'].values)), axis=1)

            X_test_final = np.concatenate((X_test_strn,X_test_resp_prod_cat, X_test_order_item,
                X_test_resp_payment_seq,X_test_payment_type,X_test_cat_price,X_test['review_availability'].values.reshape(-1,1),
                np.vstack(X_test['embedded_review_comment_message'].values)), axis=1)



            # Load the root directory from your config
            root_dir = self.config.root_dir

            # Construct file paths using the root directory
            train_data_path = os.path.join(root_dir, "train_data.json")
            test_data_path = os.path.join(root_dir, "test_data.json")

            # Combine features and label
            train_data = []
            for features, label in zip(X_train_final, y_train):
                train_data.append({'features': features.tolist(), 'label': label})

            test_data = []
            for features, label in zip(X_test_final, y_test):
                test_data.append({'features': features.tolist(), 'label': label})

            # Save to JSON files
            with open(train_data_path, "w") as f:
                json.dump(train_data, f)

            with open(test_data_path, "w") as f:
                json.dump(test_data, f)

            logger.info("Splited data into training and test sets")
            

            # Print the number of samples and features 
            num_train_samples = len(train_data)
            num_features = len(train_data[0]['features'])  #
            print(f"Train data (estimated equivalent of shape): ({num_train_samples}, {num_features})")

            num_test_samples = len(test_data)
            num_features = len(test_data[0]['features'])  #
            print(f"Test data (estimated equivalent of shape): ({num_test_samples}, {num_features})")
        

In [80]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.complete_transformation()
except Exception as e:
    raise e

[2024-03-07 17:35:31,742: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-07 17:35:31,745: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-07 17:35:31,754: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-07 17:35:31,755: INFO: common: created directory at: artifacts]
[2024-03-07 17:35:31,758: INFO: common: created directory at: artifacts/data_transformation]
[2024-03-07 17:36:28,782: INFO: SentenceTransformer: Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-mpnet-base-v2]
[2024-03-07 17:37:24,573: INFO: SentenceTransformer: Use pytorch device_name: cpu]


Batches: 100%|██████████| 624/624 [07:35<00:00,  1.37it/s]


[2024-03-07 18:15:01,055: INFO: 533007227: Splited data into training and test sets]
Train data (estimated equivalent of shape): (79787, 33)
Test data (estimated equivalent of shape): (19947, 33)
